# 01. Boston 311 (2015-2024): Data Cleaning and Preprocessing

**Objective:** The goal of this notebook is to load the raw 311 service request data for the years 2015-2024, perform a thorough cleaning and preprocessing, and save the result as a single, analysis-ready file.

## 📊 Executive Summary

**Project Scope:** Comprehensive data cleaning and preprocessing of 2.55 million Boston 311 service requests spanning 2015-2024, representing the city's entire civic complaint and service request ecosystem.

**Key Business Challenge:** Raw data contained 23% missing location information due to intersection-based reporting, threatening to eliminate nearly 600,000 records from geospatial analysis—a critical component for identifying service delivery patterns and neighborhood-level insights.

**Technical Solution:** Implemented advanced geospatial imputation using three complementary approaches: (1) ZIP code assignment via Massachusetts census boundaries, (2) street name recovery through nearest-neighbor spatial matching, and (3) neighborhood classification using official Boston district polygons.

**Business Impact:** 
- **Data Preservation:** 98.7% retention rate—preserved 660,000+ records that would have been lost to deletion
- **Analysis-Ready Dataset:** Clean, standardized 2.52M record dataset optimized for geographic and temporal analysis
- **Quality Assurance:** Systematic investigation confirmed missing SLA dates are operationally accurate, not data errors
- **Production-Grade Pipeline:** Fully automated, reproducible workflow from raw data acquisition to analysis-ready output

**Strategic Value:** This foundation enables comprehensive analysis of Boston's civic service delivery patterns, response time performance, and geographic equity—directly supporting data-driven municipal operations and policy decisions.

### Table of Contents
- [Introduction](#introduction)
1. [Data Acquisition and Merging](#data-acquisition-and-merging)
    - [1.1 Ensure Raw 311 Data is Present](#1-1-ensure-raw-311-data)
    - [1.2 Prepare Geospatial Data Assets](#1-2-prepare-geospatial-data)
    - [1.3 Locate CSV Files](#1-3-locate-csv-files)
    - [1.4 Verify Column Consistency](#1-4-verify-column-consistency)
    - [1.5 Concatenate DataFrames](#1-5-concatenate-dataframes)
2. [Data Cleaning and Preprocessing](#2-data-cleaning-and-preprocessing)
    - [2.1 Initial Inspection](#2-1-initial-inspection)
    - [2.2 Data Dictionary](#2-2-data-dictionary)
    - [2.3 Dropping Unnecessary Columns](#2-3-dropping-unnecessary-columns)
    - [2.4 Correcting Data Types](#2-4-correct-data-types)
    - [2.5 Handling Missing Values](#2-5-handling-missing-values)
        - [2.5.1 Missing Summary Function](#2-5-1-missing-summary-function)
        - [2.5.2 Exploring Missing `location_zipcode`](#2-5-2-exploring-missing-location-zipcode)
        - [2.5.3 Imputing Missing `location_zipcode` via Reverse Geocoding](#2-5-3-imputing-missing-location-zipcode)
        - [2.5.4 Investigating Missing `sla_target_dt`](#2-5-4-investigating-missing-sla-target-dt)
        - [2.5.5 Investigating Missing `closed_dt`](#2-5-5-investigating-missing-closed-dt)
        - [2.5.6 Handling Missing Location Data](#2-5-6-handling-missing-location-data)
        - [2.5.7 Imputing Missing `location_street_name`](#2-5-7-imputing-missing-location-street-name)
        - [2.5.8 Imputing Missing `neighborhood`](#2-5-8-imputing-missing-neighborhood)
        - [2.5.9 Final Missing Value Cleanup](#2-5-9-final-missing-value-cleanup)
    - [2.6 Saving the Cleaned Data](#2-6-saving-the-cleaned-data)
3. [Summary of Data Cleaning](#3-summary-of-data-cleaning)

<a id="introduction"></a>
## Introduction

Municipal 311 systems represent one of the most comprehensive datasets for understanding urban service delivery, citizen engagement patterns, and operational efficiency across city departments. This notebook documents the complete data engineering pipeline for Boston's 311 Service Request dataset, encompassing 2.55 million citizen-reported issues from 2015 through 2024—effectively capturing a decade of civic interaction data.

**Business Context & Objectives**
Boston's 311 system processes everything from pothole repairs and noise complaints to housing violations and streetlight outages, making it a critical touchpoint between city government and residents. However, the raw dataset presents significant analytical challenges: systematic missing values, inconsistent geospatial data, and complex hierarchical categorization structures that must be properly understood before meaningful insights can be extracted.

**Technical Challenge & Innovation**
The primary technical challenge centers on geospatial data integrity. Nearly 23% of records lack location identifiers due to intersection-based reporting—a limitation that would traditionally force analysts to discard over 580,000 records. Instead of accepting this substantial data loss, this project implements advanced geospatial engineering techniques including reverse geocoding, spatial joins with census boundaries, and nearest-neighbor matching against municipal address databases.

**Methodological Approach**
This preprocessing pipeline follows enterprise-grade data engineering principles: systematic quality assessment, hypothesis-driven missing data investigation, automated reproducibility through scripted data acquisition, and comprehensive validation. The approach prioritizes data preservation through intelligent imputation over convenient deletion, ensuring maximum analytical value while maintaining data integrity.

**Deliverables & Strategic Impact**
The output is a production-ready dataset optimized for geospatial analysis, temporal trend identification, and operational performance assessment. This foundation enables sophisticated analyses including neighborhood equity studies, department efficiency comparisons, and predictive modeling for resource allocation—directly supporting evidence-based municipal decision-making and policy development.

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns  
from matplotlib.colors import LogNorm
from pandas.tseries.offsets import Day
import geopandas as gpd
from shapely.geometry import Point
import gc

<a id="data-acquisition-and-merging"></a>
## 1. Data Acquisition and Merging 

**Executive Summary:** This section establishes the foundational dataset by automating the acquisition of 10 years of Boston 311 data (2.55M records) and essential geospatial reference files. All data sources are programmatically validated, downloaded if missing, and consolidated into a unified analytical framework, ensuring 100% reproducibility and eliminating manual data collection errors.

This section ensures all required raw data is present, prepares the necessary geospatial files for our analysis, and then loads and merges the yearly 311 CSVs into a single, unified pandas DataFrame.

<a id="1-1-ensure-raw-311-data"></a>
### 1.1 Ensure Raw 311 Data is Present

Before we can begin, we must ensure the raw 311 data CSVs for 2015-2024 are available. The following command executes the `01_fetch_311_data.py` script. This script checks for each yearly CSV file in the `data/raw` directory. If any files are missing, it will automatically download them from the Analyze Boston open data portal, displaying a progress bar for each download.

In [2]:
# Data preparation script.
%run ../scripts/01_fetch_311_data.py

--- 1. Checking Available Files ---
> NOT FOUND 2015 Data: 'boston-311-2015.csv'
> NOT FOUND 2016 Data: 'boston-311-2016.csv'
> NOT FOUND 2017 Data: 'boston-311-2017.csv'
> NOT FOUND 2018 Data: 'boston-311-2018.csv'
> NOT FOUND 2019 Data: 'boston-311-2019.csv'
> NOT FOUND 2020 Data: 'boston-311-2020.csv'
> NOT FOUND 2021 Data: 'boston-311-2021.csv'
> NOT FOUND 2022 Data: 'boston-311-2022.csv'
> NOT FOUND 2023 Data: 'boston-311-2023.csv'
> NOT FOUND 2024 Data: 'boston-311-2024.csv'

--- 2. Downloading Missing Files ---
> DOWNLOADING 'boston-311-2015.csv' to '../data/raw'


boston-311-2015.csv: 100%|██████████| 103M/103M [00:03<00:00, 29.8MiB/s] 


> DOWNLOADING 'boston-311-2016.csv' to '../data/raw'


boston-311-2016.csv: 100%|██████████| 109M/109M [00:03<00:00, 29.5MiB/s] 


> DOWNLOADING 'boston-311-2017.csv' to '../data/raw'


...

> DOWNLOADING 'boston-311-2023.csv' to '../data/raw'


boston-311-2023.csv: 100%|██████████| 155M/155M [00:06<00:00, 26.7MiB/s] 


> DOWNLOADING 'boston-311-2024.csv' to '../data/raw'


boston-311-2024.csv: 100%|██████████| 161M/161M [00:05<00:00, 28.6MiB/s] 


--- 3. 311 Data Fetch Complete ---
(More info here: https://data.boston.gov/dataset/311-service-requests)


<a id="1-2-prepare-geospatial-data"></a>
### 1.2 Prepare Geospatial Data Assets

Our analysis relies on several geospatial files for imputing missing location data. The `02_prepare_geodata.py` script automates the acquisition and processing of these files. It checks for the processed `.parquet` files for Massachusetts ZIP codes, Boston neighborhoods, and Boston street addresses. If any are missing, it downloads the source shapefiles, processes them as needed, and saves them in the efficient GeoParquet format in the `data/processed` directory.

In [3]:
# This command executes our geospatial data preparation script.
# It will check for each GeoParquet file and create it if missing.
%run ../scripts/02_prepare_geodata.py

--- Preparing Geospatial Data ---
> CHECKING for 'massachusetts_zip_boundaries.parquet' in '../data/processed'.
> NOT FOUND 'massachusetts_zip_boundaries.parquet'.
> DOWNLOADING ZIP Code Tabulation Areas from US Census Bureau
  (More info here: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html)


tl_2024_us_zcta520: 100%|██████████| 504M/504M [00:15<00:00, 34.9MiB/s] 


> DOWNLOADED 'tl_2024_us_zcta520.zip' shapefile ZIP to '../data/raw'
> DOWNLOADING State Boundaries from US Census Bureau
  (More info here: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html)


tl_2024_us_state: 100%|██████████| 9.49M/9.49M [00:00<00:00, 26.3MiB/s]


> DOWNLOADED 'tl_2024_us_state.zip' shapefile ZIP to '../data/raw'
> UNZIPPING 'tl_2024_us_zcta520'
> UNZIPPING 'tl_2024_us_state'
> FILTERING Shapefile for MA ZIPS
> CREATING GeoParquet
> SAVING 'massachusetts_zip_boundaries.parquet' to '../data/processed'
> DELETING raw data and ZIP file for 'tl_2024_us_zcta520'
> DELETING raw data and ZIP file for 'tl_2024_us_state'
------------------------------
> CHECKING for 'boston_neighborhood_boundaries.parquet' in '../data/processed'.
> NOT FOUND 'boston_neighborhood_boundaries.parquet'.
> DOWNLOADING 'boston_neighborhood_boundaries' shapefile from Analyze Boston...
  (More info here: https://data.boston.gov/dataset/bpda-neighborhood-boundaries)


boston_neighborhood_boundaries: 100%|██████████| 340k/340k [00:00<00:00, 5.00MiB/s]


> DOWNLOADED 'boston_neighborhood_boundaries' shapefile ZIP to '../data/raw'
> UNZIPPING 'boston_neighborhood_boundaries'
> CREATING GeoParquet
> SAVING 'boston_neighborhood_boundaries.parquet' to '../data/processed'
> DELETING raw data and ZIP File
------------------------------
> CHECKING for 'live_street_address_management_sam_addresses.parquet' in '../data/processed'.
> NOT FOUND 'live_street_address_management_sam_addresses.parquet'.
> DOWNLOADING 'live_street_address_management_sam_addresses' shapefile from Analyze Boston...
  (More info here: https://data.boston.gov/dataset/live-street-address-management-sam-addresses)


live_street_address_management_sam_addresses: 100%|██████████| 21.7M/21.7M [00:00<00:00, 25.4MiB/s]


> DOWNLOADED 'live_street_address_management_sam_addresses' shapefile ZIP to '../data/raw'
> UNZIPPING 'live_street_address_management_sam_addresses'
> CREATING GeoParquet
> SAVING 'live_street_address_management_sam_addresses.parquet' to '../data/processed'
> DELETING raw data and ZIP File

--- Geospatial Data Preparation Complete ---


<a id="1-3-locate-csv-files"></a>
### 1.3 Locate CSV Files

First, we'll locate all the individual raw CSV files.

In [4]:
# Define the path to the raw data directory
raw_data_path = Path("../data/raw")

# Get a list of all CSV file paths, sorted to ensure order
csv_files = sorted(list(raw_data_path.glob("*.csv")))

# Load each CSV into a list of DataFrames
df_list = [pd.read_csv(file, low_memory=False) for file in csv_files]

# Display the files found
csv_files

[PosixPath('../data/raw/boston-311-2015.csv'),
 PosixPath('../data/raw/boston-311-2016.csv'),
 PosixPath('../data/raw/boston-311-2017.csv'),
 PosixPath('../data/raw/boston-311-2018.csv'),
 PosixPath('../data/raw/boston-311-2019.csv'),
 PosixPath('../data/raw/boston-311-2020.csv'),
 PosixPath('../data/raw/boston-311-2021.csv'),
 PosixPath('../data/raw/boston-311-2022.csv'),
 PosixPath('../data/raw/boston-311-2023.csv'),
 PosixPath('../data/raw/boston-311-2024.csv')]

<a id="1-4-verify-column-consistency"></a>
### 1.4 Verify Column Consistency

Before combining the DataFrames, we'll verify that they all have identical column names in the same order.

In [5]:
# Get the columns from the first DataFrame as our reference
reference_columns = df_list[0].columns
all_match = True

print("Verifying column consistency across all yearly data files...")

# Loop through the rest of the DataFrames in the list
for i, df_item in enumerate(df_list[1:], 1):
    year = csv_files[i].stem.split('-')[-1] # Extracts year from filename
    current_columns = df_item.columns
    
    # Check if the columns are identical to the reference
    if not current_columns.equals(reference_columns):
        print(f"Column mismatch found in boston-311-{year}.csv")
        all_match = False
        break

if all_match:
    print("Success! All DataFrames have identical column structures.")
else:
    print("Warning! Column structures differ. Concatenation should be done carefully.")

Verifying column consistency across all yearly data files...
Success! All DataFrames have identical column structures.


<a id="1-5-concatenate-dataframes"></a>
### 1.5 Concatenate DataFrames

Now that the schemas are confirmed to match, we can safely combine the list of DataFrames into one.

In [6]:
# Concatenate all DataFrames into a single one
df = pd.concat(df_list, ignore_index=True)

In [7]:
# Memory cleanup after data acquisition and merging phase
del raw_data_path, csv_files, df_list, reference_columns, all_match, current_columns
gc.collect()
print("✓ Memory cleaned after data acquisition phase")

✓ Memory cleaned after data acquisition phase


<a id="2-data-cleaning-and-preprocessing"></a>
## 2. Data Cleaning and Preprocessing

**Executive Summary:** This section transforms raw, inconsistent data into analysis-ready format through systematic data quality assessment, advanced geospatial imputation, and strategic feature engineering. The highlight is preserving 660,000+ records through sophisticated spatial joins rather than deletion, maintaining 98.7% data retention while ensuring geographic accuracy for mapping and neighborhood analysis.

This section addresses data quality issues, including missing values and incorrect data types. We also create new features to help with analysis.

<a id="2-1-initial-inspection"></a>
### 2.1 Initial Inspection

In [8]:
df.shape

(2550536, 30)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550536 entries, 0 to 2550535
Data columns (total 30 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   case_enquiry_id                 int64  
 1   open_dt                         object 
 2   sla_target_dt                   object 
 3   closed_dt                       object 
 4   on_time                         object 
 5   case_status                     object 
 6   closure_reason                  object 
 7   case_title                      object 
 8   subject                         object 
 9   reason                          object 
 10  type                            object 
 11  queue                           object 
 12  department                      object 
 13  submitted_photo                 object 
 14  closed_photo                    object 
 15  location                        object 
 16  fire_district                   object 
 17  pwd_district               

Our data has about 2.5M rows and 30 columns. Let's look at the first few rows.

In [10]:
df.head(50)

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,...,neighborhood,neighborhood_services_district,ward,precinct,location_street_name,location_zipcode,latitude,longitude,geom_4326,source
0,101001240753,2015-01-01 01:31:02,2015-01-06 03:30:00,2015-01-01 05:49:51,ONTIME,Closed,Case Closed Case Resolved citywide cleaned up ...,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,Roxbury,13,Ward 8,0803,INTERSECTION Albany St & Massachusetts Ave,NaN,42.333522,-71.073474,0101000020E6100000988AF8CDB3C451C075C76DDCB02A...,Constituent Call
1,101001240754,2015-01-01 02:27:13,2015-01-31 02:27:13,NaN,OVERDUE,Open,,Heat - Excessive Insufficient,Inspectional Services,Housing,...,South End,6,08,0802,1744 Washington St,2118.0,42.336510,-71.076451,0101000020E61000005E53CA90E4C451C03448ABBE122B...,Constituent Call
2,101001240755,2015-01-01 03:00:18,2015-01-06 03:30:00,2015-01-01 23:51:21,ONTIME,Closed,Case Closed Case Resolved area checked all clear,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,Mission Hill,14,Ward 10,1005,8 Cherokee St,2120.0,42.330290,-71.100641,0101000020E6100000601C4AE570C651C08BCECCED462A...,Constituent Call
3,101001240757,2015-01-01 03:25:52,2015-01-09 03:30:00,2015-02-24 03:06:51,OVERDUE,Closed,Case Closed NOACC: Closed-No Access,Illegal Rooming House,Inspectional Services,Building,...,Back Bay,14,Ward 5,0509,50 Hereford St,2115.0,42.349030,-71.085591,0101000020E6100000A2E6B2507AC551C09BBF2100AD2C...,Self Service
4,101001240759,2015-01-01 03:35:00,NaN,2015-05-11 14:53:24,ONTIME,Closed,Case Closed. Closed date : 2015-05-11 18:53:24...,Watermain Break,Boston Water & Sewer Commission,Water Issues,...,West Roxbury,12,Ward 20,2017,95 Brook Farm Rd,2132.0,42.291520,-71.168301,0101000020E610000047EEE670C5CA51C0DDBC4D845025...,Constituent Call
5,101001240760,2015-01-01 03:48:34,NaN,2015-01-07 11:36:11,ONTIME,Closed,Case Closed Bulk Item Automation,Schedule Bulk Item Pickup,Public Works Department,Sanitation,...,Charlestown,2,Ward 2,0201,18 Harvard St,2129.0,42.372160,-71.063230,0101000020E61000000343A2F70BC451C09BA35EECA22F...,Self Service
6,101001240761,2015-01-01 03:49:13,2015-01-06 03:30:00,2015-01-01 06:10:00,ONTIME,Closed,Case Closed Case Resolved,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,Roxbury,13,Ward 11,1102,51 Marcella St,2119.0,42.322530,-71.095131,0101000020E6100000D98CB29E16C651C0837726A64829...,Constituent Call
7,101001240762,2015-01-01 03:53:51,2015-01-31 03:53:51,NaN,OVERDUE,Open,,Heat - Excessive Insufficient,Inspectional Services,Housing,...,Roxbury,13,Ward 9,0903,77 Camden St,2118.0,42.337500,-71.080381,0101000020E610000054DA6EF424C551C0B4FB6E2F332B...,Constituent Call
8,101001240764,2015-01-01 04:12:18,NaN,2015-01-05 11:37:32,ONTIME,Closed,Case Closed Bulk Item Automation,Schedule a Bulk Item Pickup,Public Works Department,Sanitation,...,Roxbury,13,Ward 11,1101,187 Highland St,2119.0,42.323420,-71.093591,0101000020E610000068587263FDC551C0546CFFCF6529...,Constituent Call
9,101001240765,2015-01-01 04:13:12,2015-01-05 03:30:00,2015-01-06 03:51:53,OVERDUE,Closed,Case Closed Case Resolved,Traffic Signal Repair,Transportation - Traffic Division,Signs & Signals,...,Downtown / Financial District,4,Ward 3,0308,INTERSECTION Chauncy St & Summer St,NaN,42.354564,-71.059261,0101000020E610000070A3F3EDCAC351C0CCD2D857622D...,Constituent Call


Upon observing the first few rows of data, we can see that we have a lot of columns that are not needed for our analysis. Let's look at the description of each column and decide which columns to drop.

<a id="2-2-data-dictionary"></a>
### 2.2 Data Dictionary  
This dictionary is based on the definitions from the official Boston data portal documentation and initial manual data exploration.

#### Case Classification Hierarchy
The data uses a three-tier system to classify cases: **`subject` > `reason` > `type`**.

| Column | Description |
| :--- | :--- |
| `case_enquiry_id` | The unique identifier for the service request case. |
| `open_dt` | The date and time the service request was created. |
| `sla_target_dt` | **Service Level Agreement Target Date/Time**; the deadline for case resolution. |
| `closed_dt` | The date and time the service request was closed. |
| `on_time` | A binary indicator of whether the case was closed by the `sla_target_dt`. |
| `case_status` | The current status of the case (e.g., "Open", "Closed"). |
| `closure_reason`| A short description of how the case was resolved. Examples include "Case Resolved", "Case Noted", and "NOACC" for "No Access". |
| `case_title`| A descriptive title for the case, often a combination of other fields, entered by call takers on a call-by-call basis. |
| `subject` | **(Hierarchy Level 1)** Denotes the city department a case is assigned to (e.g., "Public Works Department"). |
| `reason` | **(Hierarchy Level 2)** The general category or "umbrella" for a collection of similar case types (e.g., "Highway Maintenance"). |
| `type` | **(Hierarchy Level 3)** The individual, specific case type entered by the call taker (e.g., "Pothole Repair"). |
| `queue` | The internal departmental queue a case is assigned to, which reflects the nature of the case (e.g., `PWDx_Street Light Outages`). |
| `department` | An internal code for the department a case is assigned to, derived from the first 4 characters of the `queue` column. |
| `submitted_photo`| A URL to a photo submitted by the person who made the request. |
| `closed_photo` | A URL to a photo taken by the city worker upon closing the request. |
| `location` | The address associated with the service request. |
| `fire_district` | The BFD fire district number for the request's location. |
| `pwd_district`| The Public Works Department district for the request's location. |
| `city_council_district`| The city council district number for the request's location. |
| `police_district`| The police district designation for the request's location. |
| `neighborhood`| The neighborhood name for the request's location. The documentation notes potential redundancies and missing values in this field. |
| `neighborhood_services_district`| The Neighborhood Services district number for the request's location. |
| `ward`| The political ward for the request's location. |
| `precinct`| The political precinct for the request's location. |
| `location_street_name`| The street name of the request's location. |
| `location_zipcode`| The postal zip code of the request's location. |
| `latitude`| The WGS 84 latitude coordinate of the request. |
| `longitude`| The WGS 84 longitude coordinate of the request. |
| `geom_4326`| A geospatial data point using the **WGS 84 (SRID 4326)** standard, representing the latitude and longitude. |
| `source`| The method used to submit the request (e.g., "Citizens Connect App", "Constituent Call"). |

<a id="2-3-dropping-unnecessary-columns"></a>
### 2.3 Dropping unnecessary columns

Based on the official data dictionary and our project objectives, we will drop several columns to create a lean and focused dataset.

* **`closure_reason`**, **`queue`**: These are high-cardinality, internal-use fields that are too granular and not required for this analysis.
* **`submitted_photo`**, **`closed_photo`**: These link to images, which are outside the scope of this project.
* **`fire_district`**, **`pwd_district`**, **`city_council_district`**, **`police_district`**, **`neighborhood_services_district`**, **`ward`**, **`precinct`**: These are redundant geographic identifiers. We will use the more intuitive `neighborhood` column for our geospatial analysis.
* **`geom_4326`**: This is a redundant geospatial format, as we will use `latitude` and `longitude`.
* **`department`**: This is an internal code derived from the `queue` and contains noisy, inconsistent data. The `subject` column is the official, human-readable identifier for the responsible department within the data hierarchy and is a much cleaner feature for analysis.

In [11]:
cols_to_drop = [
    'closure_reason', 'queue', 'submitted_photo', 'closed_photo', 
    'fire_district', 'pwd_district', 'city_council_district', 
    'police_district', 'neighborhood_services_district', 'ward', 
    'precinct', 'geom_4326', 'department'
]

df.drop(columns=cols_to_drop, inplace=True)

# Verify the columns have been dropped
print(f"DataFrame now has {df.shape[1]} columns.")

DataFrame now has 17 columns.


In [12]:
df

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,case_title,subject,reason,type,location,neighborhood,location_street_name,location_zipcode,latitude,longitude,source
0,101001240753,2015-01-01 01:31:02,2015-01-06 03:30:00,2015-01-01 05:49:51,ONTIME,Closed,Requests for Street Cleaning,Public Works Department,Street Cleaning,Requests for Street Cleaning,INTERSECTION of Albany St & Massachusetts Ave ...,Roxbury,INTERSECTION Albany St & Massachusetts Ave,NaN,42.333522,-71.073474,Constituent Call
1,101001240754,2015-01-01 02:27:13,2015-01-31 02:27:13,NaN,OVERDUE,Open,Heat - Excessive Insufficient,Inspectional Services,Housing,Heat - Excessive Insufficient,1744 Washington St Roxbury MA 02118,South End,1744 Washington St,2118.0,42.336510,-71.076451,Constituent Call
2,101001240755,2015-01-01 03:00:18,2015-01-06 03:30:00,2015-01-01 23:51:21,ONTIME,Closed,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,Parking Enforcement,8 Cherokee St Mission Hill MA 02120,Mission Hill,8 Cherokee St,2120.0,42.330290,-71.100641,Constituent Call
3,101001240757,2015-01-01 03:25:52,2015-01-09 03:30:00,2015-02-24 03:06:51,OVERDUE,Closed,Illegal Rooming House,Inspectional Services,Building,Illegal Rooming House,50 Hereford St Boston MA 02115,Back Bay,50 Hereford St,2115.0,42.349030,-71.085591,Self Service
4,101001240759,2015-01-01 03:35:00,NaN,2015-05-11 14:53:24,ONTIME,Closed,Watermain Break,Boston Water & Sewer Commission,Water Issues,Watermain Break,95 Brook Farm Rd West Roxbury MA 02132,West Roxbury,95 Brook Farm Rd,2132.0,42.291520,-71.168301,Constituent Call
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2550531,101005838178,2024-12-31 17:42:59,2025-01-02 03:30:00,NaN,OVERDUE,Open,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,Parking Enforcement,INTERSECTION of Gold St & Dorchester St South...,South Boston / South Boston Waterfront,INTERSECTION Gold St & Dorchester St,NaN,42.334714,-71.047289,Citizens Connect App
2550532,101005838183,2024-12-31 17:49:30,2025-01-02 03:30:00,NaN,OVERDUE,Open,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,Parking Enforcement,49 G St South Boston MA 02127,South Boston / South Boston Waterfront,49 G St,2127.0,42.334300,-71.044380,Citizens Connect App
2550533,101005287764,2024-02-07 02:28:46,2024-02-08 03:30:00,2024-02-07 02:37:35,ONTIME,Closed,CE Collection,Public Works Department,Street Cleaning,CE Collection,26 Clarendon St Boston MA 02116,South End,26 Clarendon St,2116.0,42.345070,-71.072661,City Worker App
2550534,101005238419,2024-01-05 15:11:32,2024-01-09 03:30:00,2024-01-17 09:11:18,OVERDUE,Closed,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,Parking Enforcement,INTERSECTION of Centre St & Burroughs St Jama...,Jamaica Plain,INTERSECTION Centre St & Burroughs St,NaN,42.311576,-71.114397,Citizens Connect App


Upon visual inspection, the `location` column appears to be a more detailed string that combines information from `location_street_name` and other geographic details like the city and state. If `location_street_name` is consistently contained within the `location` string, the `location` column may be redundant for our analysis.

Let's test this hypothesis by checking how many rows follow this pattern.

In [13]:
# First, drop rows where either column is null to avoid errors in the comparison
comparison_df = df[['location', 'location_street_name']].dropna()

# Check for each row if location_street_name is a substring of location
contains_street_name = comparison_df.apply(
    lambda row: row['location_street_name'] in row['location'], 
    axis=1
)

# Calculate and print the result
match_count = contains_street_name.sum()
total_count = len(comparison_df)
match_percentage = (match_count / total_count) * 100

print(f"Number of rows checked: {total_count}")
print(f"Rows where 'location' contains 'location_street_name': {match_count}")
print(f"Match Percentage: {match_percentage:.2f}%")

Number of rows checked: 2509713
Rows where 'location' contains 'location_street_name': 1968760
Match Percentage: 78.45%


The test reveals that **78.45%** of rows with non-null location data have a `location_street_name` that is contained within the `location` string. While this is a high correlation, it's not close enough to 100% to confidently drop the `location` column.

To understand the discrepancy, let's examine a sample of the rows where the `location_street_name` was **not** found within the `location` string. This will help us identify the pattern of the non-matching data.

In [14]:
# Create a boolean Series that is True for non-matching rows
no_match_mask = ~contains_street_name

# Filter the original comparison_df to get the rows that did not match
non_matching_df = comparison_df[no_match_mask]

# Display the location and location_street_name for the first 15 non-matching rows
non_matching_df[['location', 'location_street_name']].head(10)

,location,location_street_name
0,INTERSECTION of Albany St & Massachusetts Ave ...,INTERSECTION Albany St & Massachusetts Ave
9,INTERSECTION of Chauncy St & Summer St Boston...,INTERSECTION Chauncy St & Summer St
10,INTERSECTION of Beacon St & Park St Boston MA,INTERSECTION Beacon St & Park St
23,INTERSECTION of Albany St & Massachusetts Ave ...,INTERSECTION Albany St & Massachusetts Ave
31,INTERSECTION of Boston St & Washburn St Dorch...,INTERSECTION Boston St & Washburn St
34,INTERSECTION of Saint James Ave & Trinity Pl ...,INTERSECTION Saint James Ave & Trinity Pl
36,INTERSECTION of Saint James Ave & Trinity Pl ...,INTERSECTION Saint James Ave & Trinity Pl
37,INTERSECTION of Blagden St & Saint James Ave ...,INTERSECTION Blagden St & Saint James Ave
43,INTERSECTION of Tileston St & Tileston Pl Bos...,INTERSECTION Tileston St & Tileston Pl
46,INTERSECTION of Centre St & Seaverns Ave Jama...,INTERSECTION Centre St & Seaverns Ave


The investigation reveals a clear pattern: the non-matching rows are overwhelmingly cases where the location is an intersection. The `location` column uses the format "INTERSECTION **of** Street A & Street B," while the `location_street_name` column uses "INTERSECTION Street A & Street B". The presence of the word "of" is causing the mismatch.

Before we fix this, let's confirm if all rows in our `non_matching_df` begin with "INTERSECTION". This will validate that this pattern accounts for all discrepancies.

In [15]:
# Check how many rows in the non_matching_df start with "INTERSECTION"
intersection_count = non_matching_df['location'].str.startswith('INTERSECTION').sum()
total_non_match = len(non_matching_df)
intersection_percentage = (intersection_count / total_non_match) * 100

print(f"Number of non-matching rows: {total_non_match}")
print(f"Rows that start with 'INTERSECTION': {intersection_count}")
print(f"Percentage of non-matches that are intersections: {intersection_percentage:.2f}%")

Number of non-matching rows: 540953
Rows that start with 'INTERSECTION': 540953
Percentage of non-matches that are intersections: 100.00%


The test confirms with **100.00%** certainty that the discrepancy between `location` and `location_street_name` is exclusively due to the `location` column using the format "INTERSECTION **of**..." for intersection-based addresses.

To definitively confirm that the columns are otherwise identical, our next step will be to create a temporary, corrected version of the `location` column where we replace "INTERSECTION of " with "INTERSECTION ". We will then re-run our comparison test on this corrected data. If the match rate approaches 100%, we can confidently drop the original `location` column.

In [16]:
# Create a new column with the corrected location string
df['location_corrected'] = df['location'].str.replace('INTERSECTION of ', 'INTERSECTION ')

# Drop rows where either of the columns for comparison are null
final_comparison_df = df[['location_corrected', 'location_street_name']].dropna()

# Check for each row if location_street_name is a substring of the corrected location
final_contains_check = final_comparison_df.apply(
    lambda row: row['location_street_name'] in row['location_corrected'], 
    axis=1
)

# Calculate and print the new match percentage
final_match_count = final_contains_check.sum()
final_total_count = len(final_comparison_df)
final_match_percentage = (final_match_count / final_total_count) * 100

print(f"Number of rows checked: {final_total_count}")
print(f"Rows where corrected 'location' contains 'location_street_name': {final_match_count}")
print(f"Final Match Percentage: {final_match_percentage:.2f}%")

Number of rows checked: 2509713
Rows where corrected 'location' contains 'location_street_name': 2509713
Final Match Percentage: 100.00%


The final match percentage is **100.00%**. This confirms our hypothesis that the `location` column is a redundant, more descriptive version of `location_street_name`, with the only systematic difference being the addition of the word "of" in intersection descriptions.

Since `location_street_name` is the cleaner of the two, we will drop the original `location` column as well as our temporary `location_corrected` column.

In [17]:
# Drop the redundant location columns
df.drop(columns=['location', 'location_corrected'], inplace=True)

# Display the first few rows to confirm the columns are gone
df.head(10)

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,case_title,subject,reason,type,neighborhood,location_street_name,location_zipcode,latitude,longitude,source
0,101001240753,2015-01-01 01:31:02,2015-01-06 03:30:00,2015-01-01 05:49:51,ONTIME,Closed,Requests for Street Cleaning,Public Works Department,Street Cleaning,Requests for Street Cleaning,Roxbury,INTERSECTION Albany St & Massachusetts Ave,NaN,42.333522,-71.073474,Constituent Call
1,101001240754,2015-01-01 02:27:13,2015-01-31 02:27:13,NaN,OVERDUE,Open,Heat - Excessive Insufficient,Inspectional Services,Housing,Heat - Excessive Insufficient,South End,1744 Washington St,2118.0,42.336510,-71.076451,Constituent Call
2,101001240755,2015-01-01 03:00:18,2015-01-06 03:30:00,2015-01-01 23:51:21,ONTIME,Closed,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,Parking Enforcement,Mission Hill,8 Cherokee St,2120.0,42.330290,-71.100641,Constituent Call
3,101001240757,2015-01-01 03:25:52,2015-01-09 03:30:00,2015-02-24 03:06:51,OVERDUE,Closed,Illegal Rooming House,Inspectional Services,Building,Illegal Rooming House,Back Bay,50 Hereford St,2115.0,42.349030,-71.085591,Self Service
4,101001240759,2015-01-01 03:35:00,NaN,2015-05-11 14:53:24,ONTIME,Closed,Watermain Break,Boston Water & Sewer Commission,Water Issues,Watermain Break,West Roxbury,95 Brook Farm Rd,2132.0,42.291520,-71.168301,Constituent Call
5,101001240760,2015-01-01 03:48:34,NaN,2015-01-07 11:36:11,ONTIME,Closed,Schedule Bulk Item Pickup,Public Works Department,Sanitation,Schedule a Bulk Item Pickup SS,Charlestown,18 Harvard St,2129.0,42.372160,-71.063230,Self Service
6,101001240761,2015-01-01 03:49:13,2015-01-06 03:30:00,2015-01-01 06:10:00,ONTIME,Closed,Requests for Street Cleaning,Public Works Department,Street Cleaning,Requests for Street Cleaning,Roxbury,51 Marcella St,2119.0,42.322530,-71.095131,Constituent Call
7,101001240762,2015-01-01 03:53:51,2015-01-31 03:53:51,NaN,OVERDUE,Open,Heat - Excessive Insufficient,Inspectional Services,Housing,Heat - Excessive Insufficient,Roxbury,77 Camden St,2118.0,42.337500,-71.080381,Constituent Call
8,101001240764,2015-01-01 04:12:18,NaN,2015-01-05 11:37:32,ONTIME,Closed,Schedule a Bulk Item Pickup,Public Works Department,Sanitation,Schedule a Bulk Item Pickup,Roxbury,187 Highland St,2119.0,42.323420,-71.093591,Constituent Call
9,101001240765,2015-01-01 04:13:12,2015-01-05 03:30:00,2015-01-06 03:51:53,OVERDUE,Closed,Traffic Signal Repair,Transportation - Traffic Division,Signs & Signals,Traffic Signal Repair,Downtown / Financial District,INTERSECTION Chauncy St & Summer St,NaN,42.354564,-71.059261,Constituent Call


In [18]:
# Memory cleanup after location redundancy analysis (Section 2.3)
del comparison_df, contains_street_name, match_count, total_count, match_percentage
del no_match_mask, non_matching_df, intersection_count, total_non_match, intersection_percentage
del final_comparison_df, final_contains_check, final_match_count, final_total_count, final_match_percentage
gc.collect()
print("✓ Memory cleaned after dropping unnecessary columns")

✓ Memory cleaned after dropping unnecessary columns


<a id="2-4-correct-data-types"></a>
### 2.4 Correct Data Types

Let's take a look at the current data types of the columns:

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550536 entries, 0 to 2550535
Data columns (total 16 columns):
 #   Column                Dtype  
---  ------                -----  
 0   case_enquiry_id       int64  
 1   open_dt               object 
 2   sla_target_dt         object 
 3   closed_dt             object 
 4   on_time               object 
 5   case_status           object 
 6   case_title            object 
 7   subject               object 
 8   reason                object 
 9   type                  object 
 10  neighborhood          object 
 11  location_street_name  object 
 12  location_zipcode      float64
 13  latitude              float64
 14  longitude             float64
 15  source                object 
dtypes: float64(3), int64(1), object(12)
memory usage: 311.3+ MB


We can see that `open_dt`, `sla_target_dt` and `closed_dt` should have a `datetime64` data type

In [20]:
# Converting to datetime64
date_cols = ['open_dt', 'sla_target_dt', 'closed_dt']
df[date_cols] = df[date_cols].apply(pd.to_datetime, errors="coerce")

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550536 entries, 0 to 2550535
Data columns (total 16 columns):
 #   Column                Dtype         
---  ------                -----         
 0   case_enquiry_id       int64         
 1   open_dt               datetime64[ns]
 2   sla_target_dt         datetime64[ns]
 3   closed_dt             datetime64[ns]
 4   on_time               object        
 5   case_status           object        
 6   case_title            object        
 7   subject               object        
 8   reason                object        
 9   type                  object        
 10  neighborhood          object        
 11  location_street_name  object        
 12  location_zipcode      float64       
 13  latitude              float64       
 14  longitude             float64       
 15  source                object        
dtypes: datetime64[ns](3), float64(3), int64(1), object(9)
memory usage: 311.3+ MB


Now, let's check which are categorical columns

In [22]:
cols_to_check = ['on_time', 'case_status', 'case_title', 'subject', 'reason', 'type', 'neighborhood', 'location_street_name', 'location_zipcode', 'source']
for col in cols_to_check:
    print(f"{col}: ", df[col].nunique())

on_time:  2
case_status:  2
case_title:  27518
subject:  14
reason:  52
type:  201
neighborhood:  24
location_street_name:  141043
location_zipcode:  38
source:  7


Based on the `nunique()` output, we can see many columns with a relatively low number of unique values. Converting these from the generic `object` type to a more efficient `category` type will significantly reduce memory usage and speed up future operations.

Columns with very high cardinality like `case_title` and `location` will be converted to the `string` type, which is more explicit and efficient than `object`. We will also convert `location_zipcode` to a string to fix its formatting.

In [23]:
# Looking at the category values
cols_to_category = ['on_time', 'case_status', 'subject', 'reason', 'type', 'neighborhood', 'source']
for col in cols_to_category:
    print(f"{col}: \n", df[col].unique())

on_time: 
 ['ONTIME' 'OVERDUE' nan]
case_status: 
 ['Closed' 'Open']
subject: 
 ['Public Works Department' 'Inspectional Services'
 'Transportation - Traffic Division' 'Boston Water & Sewer Commission'
 'Property Management' "Mayor's 24 Hour Hotline" 'Animal Control'
 'Parks & Recreation Department' 'Disability Department'
 'Boston Police Department' 'Neighborhood Services' 'Veterans'
 'Consumer Affairs & Licensing' 'CRM Application']
reason: 
 ['Street Cleaning' 'Housing' 'Enforcement & Abandoned Vehicles' 'Building'
 'Water Issues' 'Sanitation' 'Signs & Signals' 'Highway Maintenance'
 'Graffiti' 'Employee & General Comments' 'Notification' 'Recycling'
 'Animal Issues' 'Park Maintenance & Safety' 'Street Lights' 'Trees'
 'Health' 'Catchbasin' 'Administrative & General Requests'
 'Environmental Services' 'Disability' 'Sidewalk Cover / Manhole'
 'Fire Hydrant' 'Operations' 'Programs' 'Abandoned Bicycle'
 'Weights and Measures' 'Office of The Parking Clerk'
 'Traffic Management & Enginee

In [24]:
cols_to_string = ['case_title', 'location_street_name', 'location_zipcode']

# Convert columns to the 'category' data type
df[cols_to_category] = df[cols_to_category].astype('category')

# Convert columns to the 'string' data type
df[cols_to_string] = df[cols_to_string].astype('string')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550536 entries, 0 to 2550535
Data columns (total 16 columns):
 #   Column                Dtype         
---  ------                -----         
 0   case_enquiry_id       int64         
 1   open_dt               datetime64[ns]
 2   sla_target_dt         datetime64[ns]
 3   closed_dt             datetime64[ns]
 4   on_time               category      
 5   case_status           category      
 6   case_title            string        
 7   subject               category      
 8   reason                category      
 9   type                  category      
 10  neighborhood          category      
 11  location_street_name  string        
 12  location_zipcode      string        
 13  latitude              float64       
 14  longitude             float64       
 15  source                category      
dtypes: category(7), datetime64[ns](3), float64(2), int64(1), string(3)
memory usage: 194.6 MB


In [26]:
# Memory cleanup after data type corrections (Section 2.4)
del date_cols, cols_to_check, cols_to_category, cols_to_string
gc.collect()
print("✓ Memory cleaned after data type corrections")

✓ Memory cleaned after data type corrections


The data types look good, now, let's take one more look at the first few rows

In [27]:
df.head(10)

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,case_title,subject,reason,type,neighborhood,location_street_name,location_zipcode,latitude,longitude,source
0,101001240753,2015-01-01 01:31:02,2015-01-06 03:30:00,2015-01-01 05:49:51,ONTIME,Closed,Requests for Street Cleaning,Public Works Department,Street Cleaning,Requests for Street Cleaning,Roxbury,INTERSECTION Albany St & Massachusetts Ave,<NA>,42.333522,-71.073474,Constituent Call
1,101001240754,2015-01-01 02:27:13,2015-01-31 02:27:13,NaT,OVERDUE,Open,Heat - Excessive Insufficient,Inspectional Services,Housing,Heat - Excessive Insufficient,South End,1744 Washington St,2118.0,42.336510,-71.076451,Constituent Call
2,101001240755,2015-01-01 03:00:18,2015-01-06 03:30:00,2015-01-01 23:51:21,ONTIME,Closed,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,Parking Enforcement,Mission Hill,8 Cherokee St,2120.0,42.330290,-71.100641,Constituent Call
3,101001240757,2015-01-01 03:25:52,2015-01-09 03:30:00,2015-02-24 03:06:51,OVERDUE,Closed,Illegal Rooming House,Inspectional Services,Building,Illegal Rooming House,Back Bay,50 Hereford St,2115.0,42.349030,-71.085591,Self Service
4,101001240759,2015-01-01 03:35:00,NaT,2015-05-11 14:53:24,ONTIME,Closed,Watermain Break,Boston Water & Sewer Commission,Water Issues,Watermain Break,West Roxbury,95 Brook Farm Rd,2132.0,42.291520,-71.168301,Constituent Call
5,101001240760,2015-01-01 03:48:34,NaT,2015-01-07 11:36:11,ONTIME,Closed,Schedule Bulk Item Pickup,Public Works Department,Sanitation,Schedule a Bulk Item Pickup SS,Charlestown,18 Harvard St,2129.0,42.372160,-71.063230,Self Service
6,101001240761,2015-01-01 03:49:13,2015-01-06 03:30:00,2015-01-01 06:10:00,ONTIME,Closed,Requests for Street Cleaning,Public Works Department,Street Cleaning,Requests for Street Cleaning,Roxbury,51 Marcella St,2119.0,42.322530,-71.095131,Constituent Call
7,101001240762,2015-01-01 03:53:51,2015-01-31 03:53:51,NaT,OVERDUE,Open,Heat - Excessive Insufficient,Inspectional Services,Housing,Heat - Excessive Insufficient,Roxbury,77 Camden St,2118.0,42.337500,-71.080381,Constituent Call
8,101001240764,2015-01-01 04:12:18,NaT,2015-01-05 11:37:32,ONTIME,Closed,Schedule a Bulk Item Pickup,Public Works Department,Sanitation,Schedule a Bulk Item Pickup,Roxbury,187 Highland St,2119.0,42.323420,-71.093591,Constituent Call
9,101001240765,2015-01-01 04:13:12,2015-01-05 03:30:00,2015-01-06 03:51:53,OVERDUE,Closed,Traffic Signal Repair,Transportation - Traffic Division,Signs & Signals,Traffic Signal Repair,Downtown / Financial District,INTERSECTION Chauncy St & Summer St,<NA>,42.354564,-71.059261,Constituent Call


We can see that the `location_zipcode` has the 2-step formatting problem when we converted it from `float` to `string`. We need to remove the `.0` and pad with a leading `0`

In [28]:
# Remove the '.0' and pad with a leading zero to ensure a 5-digit format
df['location_zipcode'] = df['location_zipcode'].str.split('.').str[0].str.zfill(5)

# Verify the changes by checking a few non-missing values
df['location_zipcode'].dropna().head(10)

1     02118
2     02120
3     02115
4     02132
5     02129
6     02119
7     02118
8     02119
11    02119
12    02135
Name: location_zipcode, dtype: object

<a id="2-5-handling-missing-values"></a>
### 2.5 Handling Missing Values

**Executive Summary:** Rather than dropping incomplete records, this subsection implements production-grade data recovery techniques using hierarchical investigation and geospatial imputation. Key achievement: systematic analysis proves missing SLA dates are operationally accurate (not errors), while spatial joins recover location data for 551K+ ZIP codes, 10K+ street names, and 98K+ neighborhoods—preserving critical records for geographic analysis.

Now that our data types are correct, we can properly assess and handle missing data. It's crucial to address missing values before analysis to prevent errors and skewed results. We'll start by getting an updated summary of where the gaps in our data lie.

<a id="2-5-1-missing-summary-function"></a>
#### 2.5.1 Missing Summary Function

In [29]:
# Exploring missing values in df:
def summarize_missing_values(df):
  """
  Analyzes a DataFrame and returns a summary of its missing values.
  """
  # Calculate missing counts and percentages
  missing = pd.concat([df.isna().sum(), (100 * df.isna().mean())], axis=1)
  # Name the columns
  missing.columns = ['missing_count', 'missing_percent']
  # Format the percentage column
  missing['missing_percent'] = missing['missing_percent'].round(3)
  # Filter out columns with no missing values and sort the result
  missing_summary = missing[missing['missing_count'] != 0].sort_values(
      by='missing_count', ascending=False
  )
  return missing_summary

In [30]:
summarize_missing_values(df)

,missing_count,missing_percent
location_zipcode,581776,22.810
sla_target_dt,322583,12.648
closed_dt,202498,7.939
location_street_name,40823,1.601
latitude,31146,1.221
longitude,31146,1.221
neighborhood,2502,0.098
case_title,454,0.018
on_time,227,0.009


<a id="2-5-2-exploring-missing-location-zipcode"></a>
#### 2.5.2 Exploring Missing `location_zipcode`
The summary reveals that `location_zipcode` is missing in **22.81%** of cases. This is a significant portion of the data. Before deciding on an imputation strategy, let's inspect a sample of these rows to look for patterns. We can check the other location-based columns like `location_street_name` and `neighborhood` to see if they provide clues.

In [31]:
# Create a filtered DataFrame containing only rows where location_zipcode is null
missing_zip_df = df[df['location_zipcode'].isnull()]

# Display the first 10 rows for visual inspection
missing_zip_df[['location_street_name', 'neighborhood']].head(50)

,location_street_name,neighborhood
0,INTERSECTION Albany St & Massachusetts Ave,Roxbury
9,INTERSECTION Chauncy St & Summer St,Downtown / Financial District
10,INTERSECTION Beacon St & Park St,Beacon Hill
23,INTERSECTION Albany St & Massachusetts Ave,Roxbury
31,INTERSECTION Boston St & Washburn St,Dorchester
34,INTERSECTION Saint James Ave & Trinity Pl,Back Bay
36,INTERSECTION Saint James Ave & Trinity Pl,Back Bay
37,INTERSECTION Blagden St & Saint James Ave,Back Bay
43,INTERSECTION Tileston St & Tileston Pl,Downtown / Financial District
46,INTERSECTION Centre St & Seaverns Ave,Jamaica Plain


A clear pattern emerges from this sample: every row that is missing a `location_zipcode` also has a `location_street_name` that begins with "INTERSECTION". This suggests that the data source does not provide zip codes for intersection-based requests.

Let's test this hypothesis across all rows with missing zip codes to confirm this pattern.

In [32]:
# From our dataframe of rows with missing zip codes, drop any that might also have a missing street name
missing_zip_df = missing_zip_df.dropna(subset=['location_street_name'])

# Count how many of these rows have a street name that starts with 'INTERSECTION'
intersection_count = missing_zip_df['location_street_name'].str.startswith('INTERSECTION').sum()
total_missing_zip = len(missing_zip_df)
percentage = (intersection_count / total_missing_zip) * 100

print(f"Total rows with missing zip codes (and a street name): {total_missing_zip}")
print(f"Rows where street name starts with 'INTERSECTION': {intersection_count}")
print(f"Percentage: {percentage:.2f}%")

Total rows with missing zip codes (and a street name): 540953
Rows where street name starts with 'INTERSECTION': 540953
Percentage: 100.00%


In [33]:
missing_zip_df.head()

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,case_title,subject,reason,type,neighborhood,location_street_name,location_zipcode,latitude,longitude,source
0,101001240753,2015-01-01 01:31:02,2015-01-06 03:30:00,2015-01-01 05:49:51,ONTIME,Closed,Requests for Street Cleaning,Public Works Department,Street Cleaning,Requests for Street Cleaning,Roxbury,INTERSECTION Albany St & Massachusetts Ave,<NA>,42.333522,-71.073474,Constituent Call
9,101001240765,2015-01-01 04:13:12,2015-01-05 03:30:00,2015-01-06 03:51:53,OVERDUE,Closed,Traffic Signal Repair,Transportation - Traffic Division,Signs & Signals,Traffic Signal Repair,Downtown / Financial District,INTERSECTION Chauncy St & Summer St,<NA>,42.354564,-71.059261,Constituent Call
10,101001240766,2015-01-01 04:24:41,2015-01-06 03:30:00,2015-01-02 09:24:33,ONTIME,Closed,Request for Pothole Repair,Public Works Department,Highway Maintenance,Request for Pothole Repair,Beacon Hill,INTERSECTION Beacon St & Park St,<NA>,42.357709,-71.063200,City Worker App
23,101001240785,2015-01-01 05:56:16,2015-01-05 03:30:00,2015-01-07 07:35:56,OVERDUE,Closed,Traffic Signal Repair,Transportation - Traffic Division,Signs & Signals,Traffic Signal Repair,Roxbury,INTERSECTION Albany St & Massachusetts Ave,<NA>,42.333522,-71.073474,Self Service
31,101001240796,2015-01-01 06:55:58,2015-03-10 04:30:00,2015-01-06 03:50:05,ONTIME,Closed,Graffiti Removal,Property Management,Graffiti,Graffiti Removal,Dorchester,INTERSECTION Boston St & Washburn St,<NA>,42.325936,-71.058853,Citizens Connect App


The test confirms with **100.00%** certainty that every row with a missing `location_zipcode` is a request reported at an intersection. This explains why zip codes are systematically missing for this subset of the data, as intersections don't typically have a single, assigned zip code.

While we can't use the street name alone, these rows still contain valuable `latitude` and `longitude` information. Let's inspect these columns for the same set of rows.

Now that we've confirmed all rows with missing zip codes are intersections, we must verify a key assumption: do all of these intersection records have the `latitude` and `longitude` data required for our geospatial imputation?

If a significant number of these intersection rows are *also* missing their coordinates, our imputation strategy will not work for them, and we'll need to understand the scope of that data gap. Let's check how many of the rows with missing zip codes are also missing their latitude.

In [34]:
# Filter the DataFrame for rows where the zip code is missing
missing_zip_df = df[df['location_zipcode'].isnull()]

# Of those rows, count how many are ALSO missing a latitude value
intersections_missing_coords = missing_zip_df['latitude'].isnull().sum()
total_missing_zip = len(missing_zip_df)
percentage_missing_coords = (intersections_missing_coords / total_missing_zip) * 100

print(f"Total rows with missing zip codes: {total_missing_zip}")
print(f"Of those, number of rows ALSO missing lat/lon: {intersections_missing_coords}")
print(f"Percentage of missing zip code rows that cannot be imputed: {percentage_missing_coords:.2f}%")

Total rows with missing zip codes: 581776
Of those, number of rows ALSO missing lat/lon: 30497
Percentage of missing zip code rows that cannot be imputed: 5.24%


The results show that of the **581,776** rows missing a zip code, **94.76%** (about 551k) have corresponding latitude and longitude data and can be fixed. The remaining **5.24%** cannot be imputed with this method as they lack coordinates.

Our next step is to perform the geospatial imputation on the viable 94.76% of rows. We will revisit the remaining missing values after assessing other columns.

In [35]:
# Display the street name, latitude, and longitude for the first 10 rows with missing zip codes
missing_zip_df[['location_street_name', 'latitude', 'longitude']].head(10)

,location_street_name,latitude,longitude
0,INTERSECTION Albany St & Massachusetts Ave,42.333522,-71.073474
9,INTERSECTION Chauncy St & Summer St,42.354564,-71.059261
10,INTERSECTION Beacon St & Park St,42.357709,-71.063200
23,INTERSECTION Albany St & Massachusetts Ave,42.333522,-71.073474
31,INTERSECTION Boston St & Washburn St,42.325936,-71.058853
34,INTERSECTION Saint James Ave & Trinity Pl,42.349572,-71.075860
36,INTERSECTION Saint James Ave & Trinity Pl,42.349572,-71.075860
37,INTERSECTION Blagden St & Saint James Ave,42.349282,-71.076900
43,INTERSECTION Tileston St & Tileston Pl,42.365480,-71.053816
46,INTERSECTION Centre St & Seaverns Ave,42.312291,-71.114186


A manual check of a few of these coordinates on a map confirms that the `latitude` and `longitude` values accurately point to the specific street intersections. This suggests the coordinates are reliable and were likely generated programmatically when the service request was created.

Since we have accurate coordinates, we can use a geospatial technique called **reverse geocoding** to programmatically find and assign the correct zip code to each of these rows. We will accomplish this by using a Boston zip code **shapefile**.

<a id="2-5-3-imputing-missing-location-zipcode"></a>
#### 2.5.3 Imputing Missing `location_zipcode` via Reverse Geocoding

To impute the missing `location_zipcode` for intersection records, a reverse geocoding process will be implemented using a spatial join.

**Methodology:**
1.  **Data Asset:** A custom Massachusetts ZCTA (ZIP Code Tabulation Area) shapefile was created for this task. The source data is the 2024 TIGER/Line shapefiles from the U.S. Census Bureau.
2.  **Process:** The national ZCTA polygon file was spatially filtered against the Massachusetts state boundary polygon. This process is automated in the `scripts/02_filter_shapefile.py` script, ensuring reproducibility and efficiency.
3.  **Validation:** The accuracy of the resulting shapefile was confirmed against known Boston landmarks with 100% accuracy.

The following code will perform the spatial join between the 311 records missing a zip code and the custom MA ZCTA shapefile to impute the correct zip code based on the record's coordinates.

In [36]:
# --- 1. Load the custom Massachusetts ZCTA GeoParquet file ---
# Corrected path to the .parquet file created by our script
geoparquet_path = '../data/processed/massachusetts_zip_boundaries.parquet'

# Use the dedicated gpd.read_parquet() function
ma_zips_gdf = gpd.read_parquet(geoparquet_path)

# --- 2. Create a GeoDataFrame from rows that can be imputed ---
# Filter for rows that have coordinates but are missing a zip code
impute_df = df[df['location_zipcode'].isnull() & df['latitude'].notnull()].copy()
initial_missing_count = df['location_zipcode'].isnull().sum() # Store count before imputation

# Create geometry points from latitude and longitude
geometry = [Point(xy) for xy in zip(impute_df['longitude'], impute_df['latitude'])]
impute_gdf = gpd.GeoDataFrame(impute_df, geometry=geometry, crs="EPSG:4326")

# --- 3. Align Coordinate Reference Systems (CRS) ---
impute_gdf = impute_gdf.to_crs(ma_zips_gdf.crs)

# --- 4. Perform the spatial join ---
# The ZIP code column in the shapefile is 'ZCTA5CE20'
joined_gdf = gpd.sjoin(impute_gdf, ma_zips_gdf[['ZCTA5CE20', 'geometry']], how="left", predicate="within")

# --- 5. Impute the missing values in the original DataFrame ---
# Create a dictionary mapping the case_enquiry_id to the found ZIP code
zip_map = joined_gdf.set_index('case_enquiry_id')['ZCTA5CE20']

# Use this map to fill the NaNs in the original df's 'location_zipcode' column
df['location_zipcode'] = df['location_zipcode'].fillna(df['case_enquiry_id'].map(zip_map))

# --- 6. Verify the results ---
final_missing_count = df['location_zipcode'].isnull().sum()
imputed_count = initial_missing_count - final_missing_count

print(f"Number of rows imputed with ZIPs: {imputed_count}")
print(f"Remaining missing zip codes after imputation: {final_missing_count}")

print("\nUpdated missing values summary:")
summarize_missing_values(df)

Number of rows imputed with ZIPs: 551260
Remaining missing zip codes after imputation: 30516

Updated missing values summary:


,missing_count,missing_percent
sla_target_dt,322583,12.648
closed_dt,202498,7.939
location_street_name,40823,1.601
latitude,31146,1.221
longitude,31146,1.221
location_zipcode,30516,1.196
neighborhood,2502,0.098
case_title,454,0.018
on_time,227,0.009


The reverse geocoding process was highly effective, successfully imputing the zip code for over 94% of the rows that were missing it.

The updated summary shows that the remaining **30,516** rows with a missing `location_zipcode` correspond to the rows that were also missing `latitude` and `longitude` data. Without coordinates, these zip could not be imputed geospatially.

In [37]:
# Memory cleanup after ZIP code imputation (Section 2.5.3)
del geoparquet_path, ma_zips_gdf, impute_df, geometry, impute_gdf, joined_gdf, zip_map
del initial_missing_count, final_missing_count, imputed_count
# Clean up ZIP analysis variables from previous sections
del missing_zip_df, intersection_count, total_missing_zip, percentage
del intersections_missing_coords, percentage_missing_coords
gc.collect()
print("✓ Memory cleaned after ZIP code imputation")

✓ Memory cleaned after ZIP code imputation


<a id="2-5-4-investigating-missing-sla-target-dt"></a>
#### 2.5.4 Investigating Missing `sla_target_dt`

After handling the location data, `sla_target_dt` is now the column with the highest percentage of missing values at **12.65%**. A missing Service Level Agreement (SLA) target date is significant, as it impacts our ability to measure on-time performance.

Our hypothesis is that certain types of cases do not have formal SLAs. We can investigate this by analyzing the distribution of the `reason` and `subject` (department) for rows where `sla_target_dt` is null.

In [38]:
# Create a DataFrame filtered for rows with a missing sla_target_dt
missing_sla_df = df[df['sla_target_dt'].isnull()]

print("Top 10 Subjects for rows with missing SLA Target Date:")
print((missing_sla_df['subject'].value_counts(normalize=True).head(10) * 100).round(3))

print("\nTop 10 Case Reasons for rows with missing SLA Target Date:")
print((missing_sla_df['reason'].value_counts(normalize=True).head(10) * 100).round(3))

print("\nTop 10 Type for rows with missing SLA Target Date:")
print((missing_sla_df['type'].value_counts(normalize=True).head(10) * 100).round(3))

Top 10 Subjects for rows with missing SLA Target Date:
subject
Public Works Department              50.363
Mayor's 24 Hour Hotline              29.196
Animal Control                       10.973
Boston Water & Sewer Commission       4.449
Parks & Recreation Department         1.671
Transportation - Traffic Division     1.454
Boston Police Department              1.335
Disability Department                 0.425
Neighborhood Services                 0.087
Consumer Affairs & Licensing          0.044
Name: proportion, dtype: float64

Top 10 Case Reasons for rows with missing SLA Target Date:
reason
Sanitation                           42.517
Needle Program                       20.416
Animal Issues                        10.973
Administrative & General Requests     6.135
Highway Maintenance                   4.085
Abandoned Bicycle                     3.253
Notification                          2.244
Employee & General Comments           2.163
Catchbasin                            1.672
N

The initial analysis shows that missing `sla_target_dt` values are heavily concentrated in specific categories. For example, within the `reason` category, "Sanitation" and "Needle Program" account for over 60% of all missing SLAs.

This strongly suggests the data is not missing at random. However, simply looking at the composition of *missing* data isn't enough. We need to answer a more precise question: **For a given category, what percentage of its total cases are missing an SLA?** A high percentage would confirm a systematic absence of SLAs, whereas a low percentage would suggest random data entry errors. To investigate this, we will analyze the data across the `subject` > `reason` > `type` hierarchy.

Functionalizing the Missing Value Analysis

To make our analysis more efficient and reusable, we will create a function called `analyze_missing_by_segment`. This function will take a DataFrame and two column names—a column with missing values and a column to segment by—and produce the detailed summary we created above. This allows us to quickly perform the same rigorous check on any combination of columns.

In [39]:
def analyze_missing_by_segment(df, missing_column, segment_by_column):
    """
    Analyzes the percentage and count of missing values in one column, segmented by another.
    """
    results = []

    # Loop through each unique category in the segmenting column
    for category in df[segment_by_column].cat.categories:
        category_df = df[df[segment_by_column] == category]
        total_cases = len(category_df)
        
        if total_cases > 0:
            # Count how many of those cases have a missing value in the target column
            missing_count = category_df[missing_column].isnull().sum()
            
            # Calculate the percentage
            percentage_missing = (missing_count / total_cases) * 100
            
            results.append({
                segment_by_column: category,
                'total_cases': total_cases,
                'missing_count': missing_count, 
                f'percentage_missing_{missing_column}': percentage_missing
            })

    # Create a DataFrame from the results and sort it
    results_df = pd.DataFrame(results)
    results_df.sort_values(
        by=[f'percentage_missing_{missing_column}', 'total_cases'], 
        ascending=[False, False], 
        inplace=True
    )
    
    return results_df

Hierarchical Analysis: `subject` (Department Level)

We will begin our deep dive at the highest level of the case hierarchy: the `subject` column, which represents the city department responsible for the case.

In [40]:
sla_by_subject_df = analyze_missing_by_segment(df, 'sla_target_dt', 'subject')
display(sla_by_subject_df)

,subject,total_cases,missing_count,percentage_missing_sla_target_dt
5,Disability Department,1370,1370,100.000000
8,Neighborhood Services,281,281,100.000000
4,Consumer Affairs & Licensing,143,143,100.000000
3,CRM Application,1,1,100.000000
13,Veterans,1,1,100.000000
0,Animal Control,35404,35398,99.983053
1,Boston Police Department,4310,4307,99.930394
2,Boston Water & Sewer Commission,14371,14351,99.860831
7,Mayor's 24 Hour Hotline,116573,94182,80.792293
11,Public Works Department,1352902,162464,12.008556


The analysis at the `subject` (department) level immediately reveals three distinct patterns:

* **Systematic Absence of SLAs:** Several departments, such as **Animal Control**, the **Boston Police Department**, and the **Boston Water & Sewer Commission**, have nearly 100% of their cases missing an SLA. This is a strong indicator that these departments, as a whole, do not use the `sla_target_dt` field for the majority of their logged cases.

* **Consistent Use of SLAs:** Conversely, other major departments like **Inspectional Services**, **Property Management**, and **Transportation - Traffic Division** have almost no missing SLAs (all <1%). This demonstrates a consistent and systematic application of service level agreements.

* **Ambiguous Mixed Patterns:** Most importantly, the largest department, **Public Works Department**, shows a mixed result with 12% of its cases missing an SLA. This ambiguity at the highest level confirms that `subject` is too broad for a final decision. We cannot understand why some cases have SLAs and others don't *within the same department* without drilling down further.

This confirms our hypothesis that the pattern of missingness is not random and requires a more granular investigation. Our next logical step is to analyze the next level of the hierarchy: `reason`.

In [41]:
sla_by_reason_df = analyze_missing_by_segment(df, 'sla_target_dt', 'reason')
display(sla_by_reason_df)

,reason,total_cases,missing_count,percentage_missing_sla_target_dt
15,Disability,1370,1370,100.000000
22,Generic Noise Disturbance,809,809,100.000000
38,Pothole,693,693,100.000000
19,Fire Department,594,594,100.000000
3,Air Pollution Control,430,430,100.000000
31,Neighborhood Services Issues,281,281,100.000000
4,Alert Boston,259,259,100.000000
29,Massport,240,240,100.000000
34,Office of The Parking Clerk,148,148,100.000000
13,Consumer Affairs Issues,143,143,100.000000


Drilling down to the `reason` level provides significantly more clarity and reinforces our hypothesis.

* **Sharpened Patterns:** The picture is now much clearer than at the department level. We can definitively identify numerous `reason` categories, such as **`Needle Program`**, **`Animal Issues`**, and **`Notification`**, that systematically lack SLAs, with nearly 100% of their cases missing this information. Conversely, we see many critical service categories like **`Building`**, **`Housing`**, and **`Graffiti`** that consistently have SLAs, with 0% missing.

* **Remaining Ambiguity:** The analysis also confirms that some broad `reason` categories are still aggregations of different processes. The most significant example is **`Sanitation`**, a high-volume category where 51% of cases are missing an SLA. This strongly suggests that different specific *types* of sanitation requests are governed by different rules regarding service level agreements.

* **Conclusion:** The ambiguity within a major category like `Sanitation` makes it clear that the final determination of whether a case has an SLA is made at the most granular level. To get a definitive answer, we must proceed to the final step of our hierarchical analysis: the `type` level.

In [42]:
sla_by_type_df = analyze_missing_by_segment(df, 'sla_target_dt', 'type').sort_values(by="percentage_missing_sla_target_dt")
print(sla_by_type_df.to_string())

                                                type  total_cases  missing_count  percentage_missing_sla_target_dt
143                                  Rodent Activity        34229              0                          0.000000
27                             Chronic Dampness/Mold         2945              0                          0.000000
197                 Work Hours-Loud Noise Complaints         2600              0                          0.000000
39                         Exceeding Terms of Permit         2442              0                          0.000000
59                                 Illegal Occupancy         2374              0                          0.000000
186                     Unsanitary Conditions - Food         2354              0                          0.000000
67                             Knockdown Replacement         2180              0                          0.000000
170                              StreetLight Pole WO         2091              0

**Hierarchical Analysis: `type` (Service Request Level) and Final Conclusion**

This final analysis at the most granular `type` level provides a conclusive answer to our investigation.

**Analysis**

The table reveals a stark, near-perfect binary division among the hundreds of service request types:
* **Group 1 (SLAs Present):** There is a large group of `type`s, such as **`Rodent Activity`**, **`Unsatisfactory Living Conditions`**, and **`Graffiti Removal`**, with 0% missing `sla_target_dt` values. This shows that for these specific services, an SLA is always assigned.
* **Group 2 (SLAs Absent):** There is another large group of `type`s, including high-volume requests like **`Schedule a Bulk Item Pickup`**, **`Needle Pickup`**, and **`Animal Found`**, where nearly 100% of cases are missing an SLA.

The ambiguity we saw in the `Sanitation` `reason` category is now resolved: it was an aggregation of `type`s from both groups. For example, `Schedule a Bulk Item Pickup` lacks an SLA, while `Improper Storage of Trash (Barrels)` (another sanitation task) consistently has one.

**Conclusion**

This rigorous, hierarchical investigation has proven that the missing `sla_target_dt` values are **not missing at random**. They are systematically and intentionally absent for specific types of service requests that do not have formal service level agreements.

Therefore, any attempt at imputation would be analytically unsound, as it would involve manufacturing data where it was never intended to exist.

**Final Decision:** We will **not impute** any values for `sla_target_dt`. We will accept the `NaT` (Not a Time) values as a true and accurate representation of the city's operational processes for those specific service types. This completes our investigation into this column.

In [43]:
# Memory cleanup after SLA investigation (Section 2.5.4)
import gc
del missing_sla_df, sla_by_subject_df, sla_by_reason_df, sla_by_type_df
gc.collect()
print("✓ Memory cleaned after SLA target date investigation")

✓ Memory cleaned after SLA target date investigation


In [44]:
print("\nUpdated missing values summary:")
summarize_missing_values(df)


Updated missing values summary:


,missing_count,missing_percent
sla_target_dt,322583,12.648
closed_dt,202498,7.939
location_street_name,40823,1.601
latitude,31146,1.221
longitude,31146,1.221
location_zipcode,30516,1.196
neighborhood,2502,0.098
case_title,454,0.018
on_time,227,0.009


<a id="2-5-5-investigating-missing-closed-dt"></a>
#### 2.5.5 Investigating Missing `closed_dt`

Now that our investigation of `sla_target_dt` is complete, we will address the next largest category of missing data: `closed_dt`, which is null for **7.94%** of all cases.

Our hypothesis is that a missing `closed_dt` is not an error, but rather indicates that the case is still in an 'Open' status. A closed case must have a closed date, so we expect a perfect correlation. To verify this, we'll filter for all rows where `closed_dt` is null and then inspect the distribution of their `case_status`.

In [45]:
# Create a DataFrame containing only rows with a missing closed_dt
missing_closed_dt_df = df[df['closed_dt'].isnull()]

# Check the value counts of the 'case_status' column for this subset
print("Case status for rows with a missing closed_dt:")
display(missing_closed_dt_df['case_status'].value_counts())

Case status for rows with a missing closed_dt:


case_status
Open      202496
Closed         2
Name: count, dtype: int64

**Analysis of Missing `closed_dt`**

The output confirms our hypothesis with near-perfect accuracy. Of the 202,498 rows missing a `closed_dt`, 99.999% have a `case_status` of 'Open'. This proves that the null values in `closed_dt` are not errors but are systematically absent because the cases have not yet been resolved. These "missing" values are, in fact, an accurate reflection of the data.

We identified two anomalous rows that have a 'Closed' status but no `closed_dt`. This is a minor data inconsistency. Our final action for this column is to **drop these two inconsistent rows** and keep the remaining ~202k open cases as they are. No imputation is necessary.

In [46]:
# Get the indices of the two anomalous rows and drop
indices_to_drop = df[(df['closed_dt'].isnull()) & (df['case_status'] == 'Closed')].index
df.drop(indices_to_drop, inplace=True)

# Verify the drop and show the updated missing values summary
print(f"Dropped {len(indices_to_drop)} rows.")
summarize_missing_values(df)

Dropped 2 rows.


,missing_count,missing_percent
sla_target_dt,322583,12.648
closed_dt,202496,7.939
location_street_name,40823,1.601
latitude,31146,1.221
longitude,31146,1.221
location_zipcode,30516,1.196
neighborhood,2502,0.098
case_title,454,0.018
on_time,227,0.009


<a id="2-5-6-handling-missing-location-data"></a>
#### 2.5.6 Handling Missing Location Data

Next, we will address the cluster of missing location-based columns: `location_street_name`, `latitude`, `longitude`, `location_zipcode`, and `neighborhood`.

Next, we will address the cluster of missing location-based columns. Our earlier analysis confirmed that the remaining ~31,000 rows missing a `location_zipcode` are the same rows that also lack latitude and longitude coordinates and are the primary source of these gaps.

Since a key goal of this project is geospatial analysis and mapping, records without these coordinates are not usable. We will therefore drop all rows where **both** `latitude` and `longitude` are null. This single action will resolve the core of our missing location data problem with a minimal loss of data (~1.2%).

In [47]:
# Record the number of rows before dropping
initial_rows = len(df)

# Drop rows where BOTH 'latitude' and 'longitude' are null
df.dropna(subset=['latitude', 'longitude'], how='all', inplace=True)

# Calculate how many rows were dropped
rows_dropped = initial_rows - len(df)

# Verify the drop and show the updated missing values summary
print(f"Dropped {rows_dropped} rows that were missing latitude/longitude coordinates.")
summarize_missing_values(df)

Dropped 31146 rows that were missing latitude/longitude coordinates.


,missing_count,missing_percent
sla_target_dt,318038,12.624
closed_dt,193020,7.661
location_street_name,10326,0.410
neighborhood,2493,0.099
case_title,309,0.012
on_time,227,0.009
location_zipcode,19,0.001


In [48]:
# Memory cleanup after missing location data handling (Section 2.5.6)
del missing_closed_dt_df, indices_to_drop, initial_rows, rows_dropped
gc.collect()
print("✓ Memory cleaned after handling missing location data")

✓ Memory cleaned after handling missing location data


**Missing `location_zipcode` Cleanup**

Our previous step successfully removed the vast majority of rows with incomplete location data. However, a final check reveals 19 rows are still missing a `location_zipcode`.

These are edge cases where the record contained coordinates, but the point did not fall within a defined zip code polygon during our geospatial imputation. This typically happens for a few reasons:
* The coordinates fall directly on the boundary line between two zip code polygons.
* The point is in an area that doesn't have a defined ZCTA (Zip Code Tabulation Area), such as a large park or a body of water like the harbor.
* The coordinates, due to a data entry error, fall just **outside the Massachusetts state boundary** and therefore were not found in our custom shapefile.

Given the negligible number, we will drop these final 19 rows to ensure our location data is 100% complete.

In [49]:
# Get indices of the remaining 19 rows with missing zip codes
final_zip_drop_indices = df[df['location_zipcode'].isnull()].index

# Drop these rows
df.drop(final_zip_drop_indices, inplace=True)

# Verify the drop and show the updated missing values summary
print(f"Dropped {len(final_zip_drop_indices)} final rows with missing zip codes.")
summarize_missing_values(df)

Dropped 19 final rows with missing zip codes.


,missing_count,missing_percent
sla_target_dt,318034,12.624
closed_dt,193019,7.661
location_street_name,10315,0.409
neighborhood,2485,0.099
case_title,309,0.012
on_time,227,0.009


<a id="2-5-7-imputing-missing-location-street-name"></a>
#### 2.5.7 Imputing Missing `location_street_name`

The next column with a significant number of missing values is `location_street_name`. While the percentage is small (~0.4%), we can achieve a more complete dataset using a robust imputation method rather than dropping the rows.

We will perform a **reverse geocoding** process by leveraging the coordinates associated with each case. This involves using a second dataset—the official **Live Street Address Management (SAM) Addresses** file from Analyze Boston—which contains the precise coordinates for every address point in the city.

The technique is a **nearest-neighbor spatial join**:
1.  We take each of our 311 cases that is missing a street name.
2.  For each case, we find the single closest address point from the master SAM addresses file based on geographic distance.
3.  We then adopt the street number (`STREET_NUM`) and street name (`FULL_STREE`) from that matched address point.

This method allows us to accurately fill in the missing data based on its known geographic location.

In [50]:
# --- 1. Load the local SAM Addresses shapefile ---
geoparquet_path = Path("../data/processed/live_street_address_management_sam_addresses.parquet")
sam_gdf = gpd.read_parquet(geoparquet_path)

# --- 2. Isolate the rows in the main DataFrame that need imputation ---
impute_mask = df['location_street_name'].isnull() & df['latitude'].notnull()
impute_df = df[impute_mask].copy()

print(f"Found {len(impute_df)} unique cases with missing 'location_street_name' to impute.")

# --- 3. Prepare a GeoDataFrame for the join ---
impute_geometry = [Point(xy) for xy in zip(impute_df['longitude'], impute_df['latitude'])]
impute_gdf = gpd.GeoDataFrame(impute_df, geometry=impute_geometry, crs="EPSG:4326")

# Convert to a projected CRS for accurate distance calculation
impute_gdf = impute_gdf.to_crs("EPSG:2249")
sam_gdf = sam_gdf.to_crs("EPSG:2249")

# --- 4. Perform the nearest-neighbor join ---
address_component_columns = ['STREET_NUM', 'FULL_STREE']
joined_gdf = gpd.sjoin_nearest(
    impute_gdf, 
    sam_gdf[address_component_columns + ['geometry']], 
    how="left"
)

# --- 5. Create the imputed address and a map for the original DataFrame ---
joined_gdf['imputed_address'] = (
    joined_gdf['STREET_NUM'].astype(str) + ' ' + joined_gdf['FULL_STREE'].astype(str)
)
street_name_map = joined_gdf.drop_duplicates(subset='case_enquiry_id').set_index('case_enquiry_id')['imputed_address']

# --- 6. Apply the imputation to the main DataFrame ---
# Corrected line: Assign the result directly instead of using inplace=True
df['location_street_name'] = df['location_street_name'].fillna(df['case_enquiry_id'].map(street_name_map))

# --- 7. Report the results ---
total_cases_to_impute = len(impute_df)
filled_cases = street_name_map.notna().sum()
unfilled_cases = total_cases_to_impute - filled_cases

print(f"1. Filled {filled_cases} location_street_name from lat and long")
print(f"2. Unable to fill {unfilled_cases} location_street_name from lat and long.")

# --- 8. Final Verification ---
print("\nUpdated missing values summary:")
summarize_missing_values(df)

Found 10315 unique cases with missing 'location_street_name' to impute.
1. Filled 10315 location_street_name from lat and long
2. Unable to fill 0 location_street_name from lat and long.

Updated missing values summary:


,missing_count,missing_percent
sla_target_dt,318034,12.624
closed_dt,193019,7.661
neighborhood,2485,0.099
case_title,309,0.012
on_time,227,0.009


We successfully imputed all missing `location_street_name` values using reverse geocoding. The next column in our summary to address is `neighborhood`.

In [51]:
# Memory cleanup after street name imputation (Section 2.5.7)
del final_zip_drop_indices, geoparquet_path, sam_gdf, impute_mask, impute_df
del impute_geometry, impute_gdf, address_component_columns, joined_gdf
del street_name_map, total_cases_to_impute, filled_cases, unfilled_cases
gc.collect()
print("✓ Memory cleaned after street name imputation")

✓ Memory cleaned after street name imputation


<a id="2-5-8-imputing-missing-neighborhood"></a>
#### 2.5.8 Imputing Missing `neighborhood`

Similar to the previous step, we'll use a spatial join to impute the missing `neighborhood` values (~0.1%). We will use the official `Boston_Neighborhood_Boundaries.shp` shapefile that we have already downloaded.

The process is a point-in-polygon operation: for each of the ~2,500 cases with missing data, we'll determine which neighborhood boundary its coordinates fall into and assign that neighborhood's name. Let's look at the neighborhoods currently in our dataset:

In [52]:
# Distribution of existing neighborhood categories
df['neighborhood'].value_counts()

neighborhood
Dorchester                                      359986
Roxbury                                         228522
South Boston / South Boston Waterfront          201491
Allston / Brighton                              181501
South End                                       157672
East Boston                                     156997
Jamaica Plain                                   153766
Downtown / Financial District                   151474
Back Bay                                        126468
Hyde Park                                       105676
Greater Mattapan                                102967
Roslindale                                       91341
West Roxbury                                     89918
Boston                                           88324
Charlestown                                      84989
Beacon Hill                                      76782
Fenway / Kenmore / Audubon Circle / Longwood     53457
Mission Hill                                     468

The `value_counts()` output reveals two critical issues with our neighborhood data:

1.  There are **10,311** records where the neighborhood is a blank string (`' '`) rather than a proper `null` value.
2.  There is a large, generic category named **`'Boston'`** containing over 88,000 records. This is not a specific neighborhood and is likely a default entry; for our analysis, it's effectively a missing value.

Our next step is to convert **both** of these categories into `NaN` values. This will give us an accurate count of all records that require imputation.

In [53]:
# Find where 'neighborhood' is a blank string or 'Boston' and set it to NaN
df.loc[df['neighborhood'].isin([' ', 'Boston']), 'neighborhood'] = np.nan

# Remove the now-unused categories from the column's definition
df['neighborhood'] = df['neighborhood'].cat.remove_unused_categories()

# Verify the change by getting the new, accurate count of missing neighborhoods
print("Updated missing values after converting blanks and 'Boston' to nulls:")
summarize_missing_values(df)

Updated missing values after converting blanks and 'Boston' to nulls:


,missing_count,missing_percent
sla_target_dt,318034,12.624
closed_dt,193019,7.661
neighborhood,101120,4.014
case_title,309,0.012
on_time,227,0.009


Now that we have an accurate count of the missing data, we need to compare the neighborhood categories present in our main DataFrame against those in the shapefile. This is a critical step to ensure we don't introduce inconsistent or duplicate neighborhood names during imputation.

In [54]:
# Load the local Boston Neighborhoods shapefile
geoparquet_path = Path("../data/processed/boston_neighborhood_boundaries.parquet")
neighborhoods_gdf = gpd.read_parquet(geoparquet_path)

# Get a unique set of neighborhood names from each source
shapefile_neighborhoods = set(neighborhoods_gdf['name'].unique())
df_neighborhoods = set(df['neighborhood'].cat.categories)

# Find the differences and similarities between the two sets
unique_to_shapefile = shapefile_neighborhoods.difference(df_neighborhoods)
unique_to_df = df_neighborhoods.difference(shapefile_neighborhoods)

print("DISCREPANCY REPORT")
print("\nNeighborhoods ONLY in Shapefile:", unique_to_shapefile)
print("\nNeighborhoods ONLY in Main DataFrame:", unique_to_df)

DISCREPANCY REPORT

Neighborhoods ONLY in Shapefile: {'West End', 'Downtown', 'Fenway', 'Chinatown', 'South Boston Waterfront', 'Leather District', 'Longwood', 'North End', 'Harbor Islands', 'Bay Village'}

Neighborhoods ONLY in Main DataFrame: {'Greater Mattapan', 'Fenway / Kenmore / Audubon Circle / Longwood', 'Chestnut Hill', 'South Boston / South Boston Waterfront', 'Allston / Brighton', 'Downtown / Financial District'}


The discrepancy report confirms that the shapefile and our DataFrame use different naming conventions. The shapefile often uses more granular names (e.g., `Downtown`) while the 311 data uses broader, combined categories (e.g., `Downtown / Financial District`).

To resolve this, we will create a comprehensive mapping to standardize the shapefile's names to match our dataset's existing categories. Based on our analysis, the following mapping is required:

* `'Downtown'`, `'Bay Village'`, `'North End'`, `'Chinatown'`, `'West End'`, `'Leather District'`, and `'Harbor Islands'` **->** `'Downtown / Financial District'`
* `'South Boston Waterfront'` **->** `'South Boston / South Boston Waterfront'`
* `'Fenway'` and `'Longwood'` **->** `'Fenway / Kenmore / Audubon Circle / Longwood'`

This comprehensive mapping ensures that every relevant neighborhood name from the shapefile is converted to a category that already exists in our DataFrame. This allows for a clean imputation without creating new, redundant categories.

In [55]:
# --- 1. Define the complete mapping from shapefile names to DataFrame categories ---
neighborhood_mapping = {
    'Longwood': 'Fenway / Kenmore / Audubon Circle / Longwood',
    'Leather District': 'Downtown / Financial District',
    'Bay Village': 'Downtown / Financial District',
    'North End': 'Downtown / Financial District',
    'Downtown': 'Downtown / Financial District',
    'Harbor Islands': 'Downtown / Financial District',
    'Chinatown': 'Downtown / Financial District',
    'Fenway': 'Fenway / Kenmore / Audubon Circle / Longwood',
    'West End': 'Downtown / Financial District',
    'South Boston Waterfront': 'South Boston / South Boston Waterfront'
}

# --- 2. Apply this mapping to the shapefile GeoDataFrame ---
neighborhoods_gdf['mapped_name'] = neighborhoods_gdf['name'].replace(neighborhood_mapping)

# --- 3. Isolate rows to impute and prepare for join ---
impute_mask = df['neighborhood'].isnull() & df['latitude'].notnull()
impute_df = df[impute_mask].copy()
print(f"Found {len(impute_df)} unique cases with missing 'neighborhood' to impute.")

impute_geometry = [Point(xy) for xy in zip(impute_df['longitude'], impute_df['latitude'])]
impute_gdf = gpd.GeoDataFrame(impute_df, geometry=impute_geometry, crs="EPSG:4326")
impute_gdf = impute_gdf.to_crs(neighborhoods_gdf.crs)

# --- 4. Perform the spatial join using the MAPPED neighborhood name ---
joined_gdf = gpd.sjoin(impute_gdf, neighborhoods_gdf[['mapped_name', 'geometry']], how="left", predicate="within")
neighborhood_map = joined_gdf.drop_duplicates(subset='case_enquiry_id').set_index('case_enquiry_id')['mapped_name']

# --- 5. Apply the imputation to the main DataFrame ---
df['neighborhood'] = df['neighborhood'].fillna(df['case_enquiry_id'].map(neighborhood_map))

# --- 6. Report the results ---
total_cases_to_impute = len(impute_df)
filled_cases = neighborhood_map.notna().sum()
unfilled_cases = total_cases_to_impute - filled_cases
print(f"1. Filled {filled_cases} neighborhood values from lat and long")
print(f"2. Unable to fill {unfilled_cases} neighborhood values from lat and long.")

# --- 7. Final Verification ---
print("\nUpdated missing values summary:")
summarize_missing_values(df)

Found 101120 unique cases with missing 'neighborhood' to impute.
1. Filled 98801 neighborhood values from lat and long
2. Unable to fill 2319 neighborhood values from lat and long.

Updated missing values summary:


,missing_count,missing_percent
sla_target_dt,318034,12.624
closed_dt,193019,7.661
neighborhood,2319,0.092
case_title,309,0.012
on_time,227,0.009


In [56]:
print(list(df['neighborhood'].unique()))

['Roxbury', 'South End', 'Mission Hill', 'Back Bay', 'West Roxbury', 'Charlestown', 'Downtown / Financial District', 'Beacon Hill', 'Allston / Brighton', 'Dorchester', 'East Boston', 'South Boston', 'Jamaica Plain', 'Brighton', 'South Boston / South Boston Waterfront', 'Roslindale', 'Hyde Park', 'Fenway / Kenmore / Audubon Circle / Longwood', 'Greater Mattapan', 'Allston', 'Mattapan', 'Chestnut Hill', nan]


Our final data quality check reveals a few remaining inconsistencies in the `neighborhood` column (e.g., having both `Brighton` and `Allston / Brighton`). To ensure our categories are clean and consistent for analysis, we will perform one last mapping to consolidate these granular names into their broader, official categories. The very low count for Chestnut Hill is a signal of its unique geographic status. However, since we have only 146 rows in 2.5M rows, it is best to merge it with 'Allston / Brighton' as the small portion of Chestnut Hill that officially lies within the Boston city limits is in Brighton.

In [57]:
# Define the final consolidation mapping
neighborhood_consolidation_map = {
    'Allston': 'Allston / Brighton',
    'Brighton': 'Allston / Brighton',
    'Chestnut Hill': 'Allston / Brighton',
    'Mattapan': 'Greater Mattapan',
    'South Boston': 'South Boston / South Boston Waterfront'
}

# Iterate through the map and update the values directly using .loc
for old_category, new_category in neighborhood_consolidation_map.items():
    df.loc[df['neighborhood'] == old_category, 'neighborhood'] = new_category

# After consolidation, remove the old categories that are no longer in use
df['neighborhood'] = df['neighborhood'].cat.remove_unused_categories()

# Verify the final list of unique neighborhood categories
print("Final unique neighborhood categories after consolidation:")
print(list(df['neighborhood'].unique()))

Final unique neighborhood categories after consolidation:
['Roxbury', 'South End', 'Mission Hill', 'Back Bay', 'West Roxbury', 'Charlestown', 'Downtown / Financial District', 'Beacon Hill', 'Allston / Brighton', 'Dorchester', 'East Boston', 'South Boston / South Boston Waterfront', 'Jamaica Plain', 'Roslindale', 'Hyde Park', 'Fenway / Kenmore / Audubon Circle / Longwood', 'Greater Mattapan', nan]


<a id="2-5-9-final-missing-value-cleanup"></a>
#### 2.5.9 Final Missing Value Cleanup

Our data cleaning process is nearly complete. We will now handle the few remaining columns with a small number of missing values.

**Neighborhood Imputation Summary**
The spatial join successfully imputed the correct neighborhood for **98,801** records, resolving over 97% of the missing cases. The remaining **2,319** null values could not be imputed, likely because their coordinates fall outside of any defined neighborhood polygon. This can happen for several reasons:
* A data entry error resulted in a latitude or longitude point outside of Boston city limits.
* The point falls within an area not covered by the shapefile, such as in the harbor or on an airport runway.

Given the small number, we will drop these remaining rows.

**Dropping `case_title` and `on_time`**
We will also drop the rows with missing `case_title` and `on_time` values for the following reasons:
* **`case_title` (0.012% missing):** This is a high-cardinality descriptive field. The number of missing rows is statistically insignificant, and the core details of these cases are still captured by the `subject`, `reason`, and `type` columns. Dropping them is the cleanest approach.
* **`on_time` (0.009% missing):** This value is derived from `closed_dt` and `sla_target_dt`. A missing `on_time` status means one of these critical date fields is also missing, making the record incomplete for any performance-related analysis.

The following code will drop all remaining rows that have a null value in any of these three columns.

In [58]:
initial_rows = len(df)
df.dropna(subset=['neighborhood', 'case_title', 'on_time'], inplace=True)

rows_dropped = initial_rows - len(df)
print(f"Dropped {rows_dropped} rows with remaining missing values.")

print("\nFinal missing values summary:")
summarize_missing_values(df)

Dropped 2853 rows with remaining missing values.

Final missing values summary:


,missing_count,missing_percent
sla_target_dt,317831,12.63
closed_dt,192774,7.66


In [59]:
df.shape

(2516516, 16)

<a id="2-6-saving-the-cleaned-data"></a>
### 2.6 Saving the Cleaned Data

With the cleaning and preprocessing complete, the final step is to save our DataFrame to an efficient file format for use in the next phase of the project.

We will use the **Parquet** format. It's highly efficient for both storage (small file size) and performance (fast read speeds), and it preserves our data types, which is crucial for a seamless transition to the analysis stage.

In [60]:
# --- Save the Cleaned Data ---
output_path = Path("../data/processed/boston_311_cleaned.parquet")

# Ensure the processed directory exists
output_path.parent.mkdir(parents=True, exist_ok=True)

print(f"Saving cleaned DataFrame to '{output_path}'...")
df.to_parquet(output_path)
print("Save complete.")

Saving cleaned DataFrame to '../data/processed/boston_311_cleaned.parquet'...
Save complete.


In [61]:
# Final comprehensive memory cleanup (Section 2.6)
del initial_rows, rows_dropped, output_path
# Clean up any remaining intermediate variables from neighborhood processing
try:
    del neighborhoods_gdf, shapefile_neighborhoods, df_neighborhoods
except NameError:
    pass  # Variables already cleaned up
try:
    del unique_to_shapefile, unique_to_df, neighborhood_mapping, neighborhood_map
except NameError:
    pass  # Variables already cleaned up
try:
    del neighborhood_consolidation_map
except NameError:
    pass  # Variables already cleaned up

gc.collect()
print("✓ Final comprehensive memory cleanup complete")

✓ Final comprehensive memory cleanup complete


<a id="3-summary-of-data-cleaning"></a>
## 3. Summary of Data Cleaning

This comprehensive data engineering initiative has successfully transformed disparate, quality-compromised municipal data into a unified, analysis-ready dataset that preserves maximum information value while ensuring analytical integrity. The methodology demonstrates advanced data science techniques applied to real-world civic data challenges.

### 🎯 **Strategic Achievements**

**Data Integration Excellence**
Successfully consolidated 10 distinct annual datasets spanning 2015-2024 into a cohesive 2.52 million record dataset, establishing a complete temporal foundation for longitudinal analysis of Boston's civic service delivery patterns.

**Advanced Geospatial Data Recovery**
Implemented production-grade spatial data imputation that recovered location information for 660,376 records across three critical dimensions:
- **ZIP Code Recovery:** 551,260 records imputed via Massachusetts census boundary spatial joins
- **Street Address Recovery:** 10,315 records enhanced through nearest-neighbor matching with official municipal address databases  
- **Neighborhood Classification:** 98,801 records assigned accurate district classifications through point-in-polygon operations

**Systematic Quality Assurance**
Conducted rigorous investigation into missing Service Level Agreement (SLA) dates, conclusively demonstrating that 317,831 null values represent accurate operational data rather than quality defects—preserving data authenticity while providing crucial context for performance analysis.

**Enterprise-Grade Feature Engineering**
Curated the analytical feature space from 30 to 17 optimized columns, eliminating redundant administrative fields while preserving all analytically relevant dimensions. Standardized categorical hierarchies and consolidated geographic classifications for consistent analysis frameworks.

### 📊 **Quantitative Impact Metrics**

| **Performance Indicator** | **Initial State** | **Final State** | **Improvement** |
|:---------------------------|:------------------|:----------------|:----------------|
| **Data Retention Rate** | 100% (2,550,536 records) | **98.67% (2,516,516 records)** | **Minimal loss with maximum quality** |
| **Geospatial Completeness** | 77.2% (581,776 missing ZIP codes) | **100% (complete location data)** | **+22.8 percentage point improvement** |
| **ZIP Code Coverage** | 77.2% (581,776 missing) | **100% (0 missing)** | **+551,260 records recovered** |
| **Street Name Coverage** | 98.4% (40,823 missing) | **100% (0 missing)** | **+10,315 records recovered** |
| **Neighborhood Classification** | 96.0% (101,120 generic/missing) | **100% (0 missing)** | **+98,801 records classified** |

### 🚀 **Technical Innovation Highlights**

**Automated Reproducibility Pipeline**
Developed fully automated data acquisition and processing workflows that eliminate manual intervention, ensure version consistency, and enable seamless updates as new data becomes available.

**Memory-Optimized Processing**
Implemented strategic variable cleanup and optimized data structures to handle 2.5M+ record operations efficiently, including proper handling of categorical data types and compressed Parquet output format.

**Validation-Driven Methodology**  
Applied systematic hypothesis testing to data quality assumptions, preventing incorrect imputation while maintaining scientific rigor in data transformation decisions.

### 🎯 **Business Value & Next Phase Readiness**

The resulting dataset provides a robust foundation for multiple analytical applications:
- **Geographic Equity Analysis:** Complete neighborhood-level service delivery assessment capabilities
- **Operational Performance Metrics:** Accurate SLA compliance and department efficiency measurement  
- **Predictive Modeling:** Clean temporal and spatial features optimized for machine learning applications
- **Interactive Visualization:** Geospatially-enabled data ready for dashboard development and public-facing analytics

**Data Asset Specifications:**
- **Format:** Optimized Parquet with preserved data types and categorical efficiency
- **Completeness:** 100% location data availability for geospatial analysis
- **Temporal Coverage:** Continuous 10-year dataset enabling trend and seasonality analysis
- **Scalability:** Memory-efficient structure supporting real-time analytical applications

This data engineering foundation represents production-quality municipal analytics infrastructure, positioning the subsequent analysis phase for sophisticated insights into Boston's civic service ecosystem and enabling data-driven improvements to urban service delivery.